# Imports

In [1]:
# Standard library imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Descriptors

# Local imports

# Look at ChemBL Data

In [2]:
# Get json file path
cur_dir = os.path.dirname(os.path.realpath('__file__'))
json_path = f'{cur_dir}/data/cyp3a4_data.json'

# Load into dataframe
chembl_df = pd.read_json(json_path)

In [3]:
# Fix data
test_df = pd.DataFrame(chembl_df.to_numpy().ravel())
test_df = test_df[0].apply(pd.Series).dropna(how='all')

In [39]:
# Get rows that pertain to ic50
ic50_names = [ic50_name for ic50_name in test_df['type'].unique() if 'IC5' in ic50_name.upper()]
print(f'IC50 related names: {ic50_names}')

# Drop rows not valid for our work
test_df = test_df[
    (test_df['type'].isin(ic50_names)) & 
    (test_df['value'].notna()) &
    (test_df['value'] != 0)
]

# Fix some samples
test_df.loc[test_df['type'] == 'IC5', 'type'] = 'IC50'
print(test_df.columns)
print(test_df[(test_df['type'] == 'pIC50') & (test_df['target_pref_name'].str.contains('3A4'))])

IC50 related names: ['IC50', 'Log IC50', 'pIC50', 'log(1/IC50)', '-Log IC50(M)', 'Ratio IC50', 'IC50(app)']
Index(['action_type', 'activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units

In [2]:
# Investigate targets
unique_targets = chembl_df.target.unique()
print(
    f'Number of unique targets: {len(unique_targets)}',
    f'\nNumber of total rows: {len(chembl_df)}'
)

# We see only one target
print(f'\nTarget: {unique_targets[0]}')

# Investigate number of entries with a document listed
chembl_with_doc_df = chembl_df[
    (chembl_df.document.notnull()) &
    (chembl_df.target == unique_targets[0])
].reset_index(drop=True)
chembl_with_doc_df['original_index'] = np.arange(len(chembl_with_doc_df))
print(f'\nNumber of entries with a document attached: {len(chembl_with_doc_df)}')
chembl_with_doc_df.to_json(f'{cur_dir}/data/chembl_CYP3A4_with_doi.json', orient='records')

# Look at the supporting documents
docs = chembl_with_doc_df.document
docs_df = pd.json_normalize(docs)
print(f'Information provided by document: {docs_df.columns.tolist()}')

# Journal information (suggesting similar criteria for acceptance)
journals = docs_df.journal.dropna().unique()
print(f'\nNumber of journals within supporting documents: {len(journals)}')
print(f'Journal names: {journals}')

Number of unique targets: 1 
Number of total rows: 2394

Target: Cytochrome P450 3A4

Number of entries with a document attached: 2344
Information provided by document: ['doi', 'pmid', 'journal', 'abstract', 'stupid_response']

Number of journals within supporting documents: 11
Journal names: ['Bioorg Med Chem Lett' 'J Med Chem' 'Bioorg Med Chem' 'ACS Med Chem Lett'
 'Eur J Med Chem' 'J Nat Prod' 'Drug Metab Dispos' 'Medchemcomm'
 'Antimicrob Agents Chemother' 'RSC Med Chem' 'Med Chem Res']


In [3]:
# Extract activities and preserve original indices
activities = chembl_with_doc_df.activities.to_numpy().ravel().tolist()

# Flatten the list of activities while preserving indices
flattened_activities, original_indices, activity_indices = [], [], []
for index, sublist in enumerate(activities):
    for activity_index, item in enumerate(sublist):
        flattened_activities.append(item)
        original_indices.append(index)
        activity_indices.append(activity_index)
        
activities_df = pd.DataFrame(flattened_activities)
activities_df['original_index'] = original_indices
activities_df['activity_index'] = activity_indices

# Get the molecular weight from a smiles string
def calculate_mol_weight(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return Descriptors.ExactMolWt(mol)

# Known unit conversion factors
unit_dict = {
    'mM': 1e3,
    'nM': 1e-3
}

# Function to get unit conversion factor
def get_conversion_factor(unit, mol_weight=None):
    if unit == 'ug ml-1':
        return 1e3 / mol_weight
    elif unit == 'mg/ml':
        return 1e6 / mol_weight
    else:
        return unit_dict.get(unit)

# Units to convert to
desired_units = ['uM', '10^-6 mol/L', 'microM', 'umol/L']

# Function to convert units
def convert_units(row):
    smiles, value, unit = row['smiles'], row['value'], row['unit']
    
    if unit in desired_units:
        row['unit'] = desired_units[0]
        return row
    
    mol_weight = calculate_mol_weight(smiles)
    conversion_factor = get_conversion_factor(unit, mol_weight)
    if conversion_factor:
        row['value'] = value * conversion_factor
        row['unit'] = desired_units[0]
    
    return row

# Fix activities data
activities_df.columns = [
    'molregno', 'relation', 'value', 'unit', 
    'standard_type', 'compound_name', 'smiles',
    'original_index', 'activity_index'
]

# Allowed unit types
allowed_units = [
    'uM', '10^-6 mol/L', 'microM', 'umol/L', 
    'nM', 'mM', 'ug ml-1', # 'mg/ml'
]

# Remove incomplete rows
activities_df = activities_df[
    (activities_df['value'].notna()) &
    (activities_df['value'] != 0) &
    (activities_df['unit'].notna()) &
    (activities_df['unit'].isin(allowed_units)) &
    (activities_df['relation'].notna())
]

# Convert ic50 value to correct units
activities_df = activities_df.apply(convert_units, axis=1)

print(f'Total number of activities for target: {len(activities_df)}')

Total number of activities for target: 9316


In [5]:
# Combine data into an overall dataframe
combined_df = pd.concat([
    chembl_with_doc_df.drop(columns=['document', 'activities']), 
    docs_df
], axis=1).reset_index(drop=True)
combined_df = pd.merge(activities_df, combined_df, how='left', left_on='original_index', right_on='original_index')

print(f'Columns: {combined_df.columns.tolist()}\n')
print(combined_df.head())

Columns: ['molregno', 'relation', 'value', 'unit', 'standard_type', 'compound_name', 'smiles', 'original_index', 'activity_index', 'assay', 'target', 'target_id', 'num_activities', 'doi', 'pmid', 'journal', 'abstract', 'stupid_response']

   molregno relation  value unit standard_type  \
0   1353266        =  0.084   uM          IC50   
1   1353267        =  0.200   uM          IC50   
2   1353268        =  0.180   uM          IC50   
3   1353269        =  0.041   uM          IC50   
4   1353270        =  0.028   uM          IC50   

                                       compound_name  \
0  Thiazol-5-ylmethyl(2S,3R)-3-hydroxy-4-(N-methy...   
1  thiazol-5-ylmethyl(2S,3R)-4-(2-(3-(dimethylami...   
2  thiazol-5-ylmethyl(2S,3R)-4-(2-((2-(dimethylam...   
3  Thiazol-5-ylmethyl(2S,3R)-3-hydroxy-4-(N-isobu...   
4  Thiazol-5-ylmethyl(2S,3R)-4-(2-(ethylamino)-N-...   

                                              smiles  original_index  \
0  CN(C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)OCc1cncs1)C..

In [6]:
# Investigate dois for relational data with < or >
greater_than_df = combined_df[combined_df['relation'] == '>']
less_than_df = combined_df[combined_df['relation'] == '<']

# Compare these values to combined_df
# print(greater_than_df.value.describe())
# print(less_than_df.value.describe())

In [17]:
# Separate relational data based on previous findings
df = combined_df.copy()
df = df[
    (df['relation'] == '=')
]
df = df.sort_values(by='value', ascending=True).reset_index(drop=True)

# Save data for regression task
df = df[['original_index', 'activity_index', 'smiles', 'value', 'doi', 'journal']]
df = df[df['value'] < 1000]
df['log_value'] = df['value'].apply(np.log10)
df.to_csv(f'{cur_dir}/data/regression.csv')
df.to_json(f'{cur_dir}/data/regression.json', orient='records')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 5118 entries, 0 to 5117
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_index  5118 non-null   int64  
 1   activity_index  5118 non-null   int64  
 2   smiles          5118 non-null   object 
 3   value           5118 non-null   float64
 4   doi             5118 non-null   object 
 5   journal         5028 non-null   object 
 6   log_value       5118 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 448.9+ KB


,original_index,activity_index,smiles,value,doi,journal,log_value
0,53,7,CC(C)[C@@H]1COC[C@H](C2(OC(=O)N3CCNCC3)CC2)N1S...,0.0003,10.1016/j.bmcl.2010.09.028,Bioorg Med Chem Lett,-3.522879
1,51,1,Cc1cc(N2CCC(O)CC2)cc2[nH]c(-c3c(NC[C@@H](O)c4c...,0.0004,10.1016/j.bmcl.2008.05.104,Bioorg Med Chem Lett,-3.397940
2,649,3,O=C(NC1CCC(=Cc2cccc(Oc3ccc(C(F)(F)F)cn3)c2)CC1...,0.0004,10.1016/j.bmcl.2018.11.048,Bioorg Med Chem Lett,-3.397940
3,53,2,CC(C)[C@@H]1COC[C@H](C2(OC(=O)N3CC4CCC(C3)N4)C...,0.0005,10.1016/j.bmcl.2010.09.028,Bioorg Med Chem Lett,-3.301030
4,51,0,Cc1cc(N2CCOCC2)cc2[nH]c(-c3c(NC[C@@H](O)c4cccc...,0.0005,10.1016/j.bmcl.2008.05.104,Bioorg Med Chem Lett,-3.301030


In [13]:
# Look at the number of entries per journal
print('Number of entries for journal:')
for journal in journals:
    print(f'\t{journal}: {len(df[df["journal"] == journal])}')

Number of entries for journal:
	Bioorg Med Chem Lett: 2220
	J Med Chem: 1917
	Bioorg Med Chem: 247
	ACS Med Chem Lett: 282
	Eur J Med Chem: 189
	J Nat Prod: 61
	Drug Metab Dispos: 80
	Medchemcomm: 24
	Antimicrob Agents Chemother: 2
	RSC Med Chem: 6
	Med Chem Res: 0


In [15]:
print(df.drop_duplicates(['smiles', 'value']))

      original_index  activity_index  \
0                 53               7   
1                 51               1   
2                649               3   
3                 53               2   
4                 51               0   
...              ...             ...   
5113             283               7   
5114             915               0   
5115             283               6   
5116            1290               0   
5117             283               9   

                                                 smiles     value  \
0     CC(C)[C@@H]1COC[C@H](C2(OC(=O)N3CCNCC3)CC2)N1S...    0.0003   
1     Cc1cc(N2CCC(O)CC2)cc2[nH]c(-c3c(NC[C@@H](O)c4c...    0.0004   
2     O=C(NC1CCC(=Cc2cccc(Oc3ccc(C(F)(F)F)cn3)c2)CC1...    0.0004   
3     CC(C)[C@@H]1COC[C@H](C2(OC(=O)N3CC4CCC(C3)N4)C...    0.0005   
4     Cc1cc(N2CCOCC2)cc2[nH]c(-c3c(NC[C@@H](O)c4cccc...    0.0005   
...                                                 ...       ...   
5113    Cn1c(N2CCO[C@@H]3CCC[C@H]32)